# 在Bedrock上微调Claude 3 Haiku
在本笔记本中，我们将引导您完成在Amazon Bedrock上微调Claude 3 Haiku的过程

## 您需要准备的资源
- 具有Bedrock访问权限的AWS账户
- 一个数据集（或您可以使用此处提供的示例数据集）
- [一个能够访问保存训练数据的S3存储桶的服务角色](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-iam-role.html)

## 安装依赖项

In [ ]:
!pip install boto3

In [ ]:
import boto3

## 准备数据集
您用于Bedrock微调的数据集需要是一个JSONL文件（即每行包含一个json对象的文件）。

JSONL文件中的每一行都应该是一个具有以下结构的JSON对象：

```
{
  "system": "<可选的系统消息>",
  "messages": [
    {"role": "user", "content": "用户消息"},
    {"role": "assistant", "content": "助手回复"},
    ...
  ]
}
```

- `system`字段是可选的。
- 至少需要两条消息。
- 第一条消息必须来自"user"。
- 最后一条消息必须来自"assistant"。
- 用户和助手消息必须交替出现。
- 不允许使用额外的键。


## 示例数据集 - JSON模式
我们包含了一个示例数据集，用于教授模型以JSON格式回答所有问题。以下是该数据集的样式：

In [ ]:
import json

sample_dataset = []
dataset_path = "datasets/json_mode_dataset.jsonl"
with open(dataset_path, "r") as f:
    for line in f:
        sample_dataset.append(json.loads(line))

print(json.dumps(sample_dataset[0], indent=2))

## 将您的数据集上传到S3
您用于微调的数据集应该在S3上可用；在本演示中，我们将示例数据集写入您控制的S3存储桶中

In [ ]:
bucket_name = "YOUR_BUCKET_NAME"
s3_path = "json_mode_dataset.jsonl"

s3 = boto3.client("s3")
s3.upload_file(dataset_path, bucket_name, s3_path)

## 启动Bedrock微调作业

现在您已经准备好数据集，您可以使用`boto3`启动微调作业。首先，我们将为作业配置一些参数：

In [ ]:
# 配置
job_name = "anthropic-finetuning-cookbook-training"
custom_model_name = "anthropic_finetuning_cookbook"
role = "YOUR_AWS_SERVICE_ROLE_ARN"
output_path = f"s3://{bucket_name}/finetuning_example_results/"
base_model_id = (
    "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-haiku-4-5-20251001-v1:0:200k"
)

# 超参数
epoch_count = 5
batch_size = 4
learning_rate_multiplier = 1.0

然后我们可以使用`boto3`启动作业

In [ ]:
bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

bedrock.create_model_customization_job(
    customizationType="FINE_TUNING",
    jobName=job_name,
    customModelName=custom_model_name,
    roleArn=role,
    baseModelIdentifier=base_model_id,
    hyperParameters={
        "epochCount": f"{epoch_count}",
        "batchSize": f"{batch_size}",
        "learningRateMultiplier": f"{learning_rate_multiplier}",
    },
    trainingDataConfig={"s3Uri": f"s3://{bucket_name}/{s3_path}"},
    outputDataConfig={"s3Uri": output_path},
)

您可以使用此功能在模型训练时检查作业状态：

In [ ]:
# 检查作业状态
status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]

## 使用您的微调模型！

要使用您的微调模型，[您需要使用Amazon Bedrock中的配置吞吐量(Provisioned Throughput)托管它](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-use.html)。一旦您的模型通过配置吞吐量准备就绪，您就可以通过Bedrock API调用您的模型。

In [ ]:
provisioned_throughput_arn = "YOUR_PROVISIONED_THROUGHPUT_ARN"

In [ ]:
bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "system": "JSON模式：已启用",
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": "什么是大型语言模型？"}],
            }
        ],
    }
)
response = bedrock_runtime.invoke_model(modelId=provisioned_throughput_arn, body=body)
body = json.loads(response["body"].read().decode("utf-8"))

In [ ]:
print(body["content"][0]["text"])